In [ ]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array

# load doc into memory

def load_doc(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# split a loaded document into sentences
def to_pairs(doc):
	lines = doc.strip().split('\n')
	pairs = [line.split('\t') for line in  lines]
	return pairs

# clean a list of lines
def clean_pairs(lines):
	cleaned = list()
	# prepare regex for char filtering

	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for pair in lines:
		clean_pair = list()
		for line in pair:

			# tokenize on white space
			line = line.split()

			# remove punctuation from each token
			line = [word.translate(table) for word in line]

			# store as string
			clean_pair.append(' '.join(line))
		cleaned.append(clean_pair)
	return array(cleaned)

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load dataset
filename = '/content/aze.txt'
doc = load_doc(filename)
# split into english-german pairs
pairs = to_pairs(doc)
# clean sentences
clean_pairs = clean_pairs(pairs)
# save clean pairs to file
save_clean_data(clean_pairs, 'english-Azerbaijani.pkl')
# spot check
for i in range(100):
	print('[%s] => [%s]' % (clean_pairs[i,0], clean_pairs[i,1]))

Saved: english-Azerbaijani.pkl
[Im OK] => [Mən yaxşıyam]
[Really] => [Əminsən]
[Really] => [Doğurdan]
[Really] => [Dəqiq]
[Its me] => [Bu mənəm]
[Join us] => [Bizə qoşul]
[Perfect] => [Əla]
[Perfect] => [Möhtəşəm]
[Shut up] => [Kəs səsini]
[Shut up] => [Yum ağzını]
[Tom won] => [Tom uddu]
[Welcome] => [Xoş gəlmişsiniz]
[You won] => [Sən uddun]
[Hurry up] => [Tələs]
[Im home] => [Evdəyəm]
[Im ugly] => [Mən çirkinəm]
[Im ugly] => [Mən eybəcərəm]
[Tom came] => [Tom gəldi]
[Whos he] => [O kimdir]
[Birds fly] => [Quşlar uçurlar]
[I knew it] => [Mən bunu bilirdim]
[I laughed] => [Mən güldüm]
[Ill talk] => [Mən danışaram]
[Im angry] => [Hirsliyəm]
[Im tired] => [Yorulmuşam]
[Im tired] => [Yorğunam]
[Im young] => [Mən gəncəm]
[Im young] => [Mən cavanam]
[Is it bad] => [Pisdir]
[Is it hot] => [Qaynardır]
[Well win] => [Biz qalib gələcəyik]
[Be careful] => [Ehtiyatlı olun]
[Be careful] => [Ehtiyatlı ol]
[Be serious] => [Ciddi olun]
[Be serious] => [Ciddi ol]
[Birds sing] => [Quşlar oxuyurlar]
[D

In [ ]:
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load dataset
raw_dataset = load_clean_sentences('english-Azerbaijani.pkl')

# reduce dataset size
n_sentences = 2192
dataset = raw_dataset[:n_sentences, :]
# random shuffle
shuffle(dataset)
# split into train/test
train, test = dataset[:2000], dataset[192:]
# save
save_clean_data(dataset, 'english-Azerbaijani-both.pkl')
save_clean_data(train, 'english-Azerbaijani-train.pkl')
save_clean_data(test, 'english-Azerbaijani-test.pkl')

Saved: english-Azerbaijani-both.pkl
Saved: english-Azerbaijani-train.pkl
Saved: english-Azerbaijani-test.pkl


In [ ]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

# one hot encode target sequence
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(LSTM(n_units))
	model.add(RepeatVector(tar_timesteps))
	model.add(LSTM(n_units, return_sequences=True))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	return model

# load datasets
dataset = load_clean_sentences('english-Azerbaijani-both.pkl')
train = load_clean_sentences('english-Azerbaijani-train.pkl')
test = load_clean_sentences('english-Azerbaijani-test.pkl')

# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:,0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
print(' Azerbaijani Vocabulary Size: %d' % ger_vocab_size)
print(' Azerbaijani Max Length: %d' % (ger_length))

# prepare training data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = encode_output(trainY, eng_vocab_size)
# prepare validation data
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
testY = encode_output(testY, eng_vocab_size)

# define model
model = define_model(ger_vocab_size, eng_vocab_size, ger_length, eng_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')
# summarize defined model
print(model.summary())
plot_model(model, to_file='model.png', show_shapes=True)
# fit model
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=100, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)

English Vocabulary Size: 1854
English Max Length: 16
 Azerbaijani Vocabulary Size: 3206
 Azerbaijani Max Length: 13
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 13, 256)           820736    
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 16, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 16, 256)           525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 16, 1854)          476478    
Total params: 2,347,838
Trainable params: 2,347,838
Non-trainable params: 0
____________________________________________________________

In [ ]:
from pickle import load
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, eng_tokenizer, source)
		raw_target, raw_src,test = raw_dataset[i]
		if i < 10:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
		actual.append([raw_target.split()])
		predicted.append(translation.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

# load datasets
dataset = load_clean_sentences('english-Azerbaijani-both.pkl')
train = load_clean_sentences('english-Azerbaijani-train.pkl')
test = load_clean_sentences('english-Azerbaijani-test.pkl')
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
# prepare data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])

# load model
model = load_model('model.h5')
# test on some training sequences
print('train')
evaluate_model(model, eng_tokenizer, trainX, train)
# test on some test sequences
print('test')
evaluate_model(model, eng_tokenizer, testX, test)

train
src=[Kopenhagen Danimarkanın paytaxtıdır], target=[Copenhagen is the capital of Denmark], predicted=[copenhagen is the capital of denmark]
src=[Pul istəyirəm], target=[I want money], predicted=[i want want]
src=[Tom Meridən qorxmur], target=[Tom isnt afraid of Mary], predicted=[tom is afraid of china]
src=[Pişik stolun üstündə oturdu], target=[The cat sat on the table], predicted=[the cat sat on the table]
src=[Bu alma turş dadır], target=[This apple tastes sour], predicted=[this apple tastes sour]
src=[Gəl çarpayının altına baxaq], target=[Lets look under the bed], predicted=[lets look under the bed]
src=[Papua Yeni Qvineya alman koloniyası idi], target=[Papua New Guinea was a German colony], predicted=[papua of guinea was a german colony]
src=[Hər yerdə qan var idi], target=[There was blood everywhere], predicted=[there was blood everywhere]
src=[Tom taksi sürücüsü oldu], target=[Tom became a taxi driver], predicted=[tom became a taxi driver]
src=[Bütün şəhər suyun altında idi]